In [ ]:
import panel as pn
import param
import holoviews as hv,panel as pn,param
import holoviews as hv
from holoviews import streams
hv.extension('bokeh')
from holoviews.operation.datashader import datashade
import numpy as np
from panel.pane import LaTeX


In [ ]:
def Nesterov(p,h,alpha,Nb,x0):
    x=np.zeros(Nb)
    x[0]=x0
    x[1]=x0-h*p*(x[0])**(p-1)
    y=np.copy(x)
    for k in range(1,Nb-1):
        y[k+1]=x[k]+(k/(k+alpha))*(x[k]-x[k-1])
        x[k+1]=y[k+1]-h*p*np.sign(y[k+1])*(np.abs(y[k+1])**(p-1))
    return x

In [ ]:
def rate(alpha,p):
    if p<=2:
        return 2*alpha*p/(p+2)
    if p>2:
        return min(2*alpha*p/(p+2),2*p/(p-2))

In [ ]:
PMAX=8
ALPHAMAX=7
p= np.arange(1, PMAX,0.01)
alpha = np.arange(0, ALPHAMAX, 0.01)
A,P= np.meshgrid(alpha,p)
vrate=np.vectorize(rate)
R=vrate(A,P)
Rate = hv.Dataset((alpha, p,R),['alpha', 'p'], 'rate')
RateVisu=Rate.to(hv.Image, ['alpha', 'p'])
C1=RateVisu.sample(p=2)
hv.Curve(C1).redim.range(alpha=(0, ALPHAMAX),rate=(0, 4.0))
#print(C1)

In [ ]:
alpha=3
p=3

In [ ]:
def nesterov(alpha,p,x0=1,step=0.0000005,T = 70):
    Nb=int(np.floor(T/np.sqrt(step)))
    xnest=Nesterov(p,step,alpha,Nb,x0)
    sousech = 10
    xrescale=np.abs(xnest[0::sousech])**p*np.arange(0,Nb,sousech)**rate(alpha,p)
    xrescale = xrescale/np.max(xrescale)
    n1=len(xrescale)
    time=np.linspace(0,T,n1)
    pt="%2.2f" % rate(alpha,p)
    #pt=str(rate(alpha,p))
    stral=str(alpha)
    strp=str(p)
    te1='alpha='+stral+', p='+strp
    formula='$\alpha$'
    te2='Expected rate : '+pt
    TN=pn.Column(LaTeX(te1,size=25,dpi=100),LaTeX(te2,size=25,dpi=100))
    #ht=hv.Text(T/2, 0.8, 'Expected rate : '+pt).opts(text_font_size='18pt')
    h = hv.Scatter(xnest[0::10],kdims='n',vdims = 'fonctionelle').options(width=600)
    h2 = pn.Row(hv.Scatter((time,xrescale),kdims='time',vdims = 'rescale').options(width=600),TN)
    return h2

In [ ]:
p=3
alpha=4
nesterov(alpha,p,x0=1,step=0.000005,T = 1400)

In [ ]:
p=3
alpha=1
nesterov(alpha,p,x0=1,step=0.0000005,T = 50)

In [ ]:
p=5
alpha=1
nesterov(alpha,p,x0=1,step=0.0000005,T = 100)

In [ ]:
p=2
alpha=3
nesterov(alpha,p,x0=1,step=0.0000001,T = 20)

In [ ]:
p=1.5
alpha=3
nesterov(alpha,p,x0=1,step=0.00000001,T = 15)